In [ ]:
!git clone https://github.com/yeaung276/FaceRecognization.git

In [ ]:
!pip install tfx

In [3]:
!mkdir images && cd images

In [ ]:
!wget https://facesyntheticspubwedata.blob.core.windows.net/wacv-2023/subjects_0-1999_72_imgs.zip

In [ ]:
!unzip subjects_0-1999_72_imgs.zip -d /content/images
!rm /content/subjects_0-1999_72_imgs.zip

In [2]:
%cd /content/FaceRecognization

/content/FaceRecognization


In [3]:
!pwd

/content/FaceRecognization


In [4]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.orchestration import metadata
from pipeline.example_gen import TripletExampleGen
from tfx.proto import example_gen_pb2

Using TensorFlow backend


In [5]:
context = InteractiveContext(
    pipeline_root='/content/root',
    metadata_connection_config=metadata.sqlite_metadata_connection_config('/content/metadata')
    )

In [6]:
example_gen = TripletExampleGen(
    input_base='/content/images',
    sample_per_class=7,
    eval_split_ratio=0.2
    )
context.run(example_gen)

ExecutionResult(
    component_id: TripletExampleGen
    execution_id: 4
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=TripletExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [7]:
from tfx.dsl.components.common.importer import Importer
from tfx.types import standard_artifacts

model = Importer(
    source_uri='models/base-models/mobile-net',
    artifact_type=standard_artifacts.Model
  ).with_id('model_importer')

context.run(model)

ExecutionResult(
    component_id: model_importer
    execution_id: 5
    outputs:
        result: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "models/base-models/mobile-net"
        custom_properties {
          key: "is_external"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.14.0"
          }
        }
        type: "Model"
        create_time_since_epoch: 1699265117774
        last_update_time_since_epoch: 1699265139512
        , artifact_type: id: 16
        name: "Model"
        base_type: MODEL
        )]
            additional_properties: {}
            additional_custom_properties: {}
            _input_trigger: None
        ))

In [8]:
from pipeline.embedding_gen import EmbeddingGen

encoding_gen = EmbeddingGen(
      examples=example_gen.outputs['examples'],
      model=model.outputs['result']
    )
context.run(encoding_gen)

ExecutionResult(
    component_id: EmbeddingGen
    execution_id: 6
    outputs:
        output: OutputChannel(artifact_type=Examples, producer_component_id=EmbeddingGen, output_key=output, additional_properties={}, additional_custom_properties={}, _input_trigger=None)